<a href="https://colab.research.google.com/github/Kerriea-star/Deep-Learning-Time-Series/blob/main/Deep_Learning_for_Time_Series_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install chart-studio

In [2]:
!pip install --upgrade tensorflow

In [9]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import chart_studio.plotly as py
import plotly.graph_objs as go

from plotly.offline import init_notebook_mode, iplot

%matplotlib inline

warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

# Set seeds to make the experiment more reproducible.
import tensorflow as tf
from numpy.random import seed

tf.random.set_seed(1)
seed(1)

In [13]:
train = pd.read_csv("drive/MyDrive/Colab Notebooks/Time Series Forecasting/train.csv", parse_dates=['date'])
test = pd.read_csv("drive/MyDrive/Colab Notebooks/Time Series Forecasting/test.csv", parse_dates=['date'])

In [14]:
train.describe()

,store,item,sales
count,913000.000000,913000.000000,913000.000000
mean,5.500000,25.500000,52.250287
std,2.872283,14.430878,28.801144
min,1.000000,1.000000,0.000000
25%,3.000000,13.000000,30.000000
50%,5.500000,25.500000,47.000000
75%,8.000000,38.000000,70.000000
max,10.000000,50.000000,231.000000


In [15]:
train.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [16]:
train.tail()

,date,store,item,sales
912995,2017-12-27,10,50,63
912996,2017-12-28,10,50,59
912997,2017-12-29,10,50,74
912998,2017-12-30,10,50,62
912999,2017-12-31,10,50,82


In [17]:
print("Min date from train set: %s" % train['date'].min().date())
print("Max date from train set: %s" % train['date'].max().date())

Min date from train set: 2013-01-01
Max date from train set: 2017-12-31


In [18]:
lag_size = (test['date'].max().date() - train["date"].max().date()).days
print("Max date from train set: %s" % train["date"].max().date())
print("Max date from test set: %s" % test["date"].max().date())
print("Forecast lag size", lag_size)

Max date from train set: 2017-12-31
Max date from test set: 2018-03-31
Forecast lag size 90


In [19]:
daily_sales = train.groupby("date", as_index=False)["sales"].sum()
store_daily_sales = train.groupby(["store", "date"], as_index=False)["sales"].sum()
item_daily_sales = train.groupby(["item", "date"], as_index=False)["sales"].sum()


In [24]:
daily_sales_sc = go.Scatter(x=daily_sales["date"], y=daily_sales["sales"])
layout = go.Layout(title="Daily sales", xaxis=dict(title="Date"), yaxis=dict(title="Sales"))
fig = go.Figure(data=[daily_sales_sc], layout=layout)
iplot(fig)

In [26]:
store_daily_sales_sc = []
for store in store_daily_sales['store'].unique():
  current_store_daily_sales = store_daily_sales[(store_daily_sales["store"] == store)]
  store_daily_sales_sc.append(go.Scatter(x=current_store_daily_sales["date"], y=current_store_daily_sales["sales"], name=("Store %s" % store)))

layout = go.Layout(title="Store daily sales", xaxis=dict(title="Date"), yaxis=dict(title="Sales"))
fig = go.Figure(data=store_daily_sales_sc, layout=layout)
iplot(fig)